In [19]:
import ee 
import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [20]:
#Initialize Google Earth Engine
ee.Initialize(project = 'geoai-deforestation')
print("GEE is ready!")

GEE is ready!


In [21]:

# Define Amazon region (1°×1° for speed)
geometry = ee.Geometry.Rectangle([-57, -8, -56, -7])

In [22]:
# Cloud masking function
def maskClouds(image):
    scl = image.select('SCL')
    mask = scl.eq(4)  # Vegetation only for cleaner NDVI
    return image.updateMask(mask)

In [23]:
# Fetch Sentinel-2 imagery (2022-2023, cloud-free, Level-2A Harmonized)
try:
    s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(geometry) \
        .filterDate('2022-01-01', '2023-12-31') \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .map(maskClouds) \
        .median()
    s2_bands = s2.select(['B4', 'B8'])
    print("Sentinel-2 SR Harmonized fetched!")
except Exception as e:
    print(f"Failed to fetch Sentinel-2: {e}")
    raise

Sentinel-2 SR Harmonized fetched!


In [24]:
# Fetch GFW tree cover loss (2023)
try:
    gfw = ee.Image('UMD/hansen/global_forest_change_2023_v1_11') \
        .select('lossyear') \
        .eq(23)  # 2023 loss (23 = 2023)
    print("GFW fetched!")
except Exception as e:
    print(f"Failed to fetch GFW: {e}")
    raise

GFW fetched!


In [25]:
# Create output directory
data_dir = 'data'
os.makedirs(data_dir, exist_ok=True)

In [26]:
# Export Sentinel-2 to GeoTIFF
try:
    task_s2 = ee.batch.Export.image.toDrive(
        image=s2_bands,
        description='sentinel2_sr_amazon_2023',
        folder='geoai_deforestation',
        region=geometry.getInfo()['coordinates'],
        scale=10,
        crs='EPSG:4326',
        fileFormat='GeoTIFF',
        maxPixels=2e8  # Increased to 200 million
    )
    task_s2.start()
    print("Exporting Sentinel-2 SR to Google Drive... Check Drive for 'sentinel2_sr_amazon_2023.tif'.")
except Exception as e:
    print(f"Sentinel-2 export failed: {e}")
    raise

Exporting Sentinel-2 SR to Google Drive... Check Drive for 'sentinel2_sr_amazon_2023.tif'.


In [27]:
# Export GFW to GeoTIFF
try:
    task_gfw = ee.batch.Export.image.toDrive(
        image=gfw,
        description='gfw_loss_2023',
        folder='geoai_deforestation',
        region=geometry.getInfo()['coordinates'],
        scale=30,
        crs='EPSG:4326',
        fileFormat='GeoTIFF',
        maxPixels=2e8  # Increased for safety
    )
    task_gfw.start()
    print("Exporting GFW to Google Drive... Check Drive for 'gfw_loss_2023.tif'.")
except Exception as e:
    print(f"GFW export failed: {e}")
    raise

Exporting GFW to Google Drive... Check Drive for 'gfw_loss_2023.tif'.
